In [38]:
import freud.box
import pandas as pd
from freud import density, box
import numpy as np
import matplotlib.pyplot as plt
from scipy import spatial
from shapely.geometry import Polygon, Point, MultiPoint
%matplotlib auto

Using matplotlib backend: Qt5Agg


In [2]:
data = pd.read_hdf("/media/data/Data/BallBearing/HIPS/IslandExperiments/1,91mmRepeatsB/19400010.hdf5")

In [413]:
def corr(features, density, radius, r_min, r_max, dr):
    N = features.x.count()
    dists, orders, N = dists_and_orders(features, r_max * radius)
    r_values = np.arange(r_min, r_max, dr) * radius

    g, bins = np.histogram(dists, bins=r_values)
    g6, bins = np.histogram(dists, bins=r_values, weights=orders)
    bin_centres = bins[1:] - (bins[1] - bins[0]) / 2
    divisor = 2 * np.pi * bin_centres * (bins[1] - bins[0]) * density * N

    g = g / divisor
    g6 = g6 / divisor
    return bin_centres, g, g6

def corr_custom_bins(features, bins, density, radius, r_min, r_max, dr):
    N = features.x.count()
    dists, orders, N = dists_and_orders(features, r_max*radius)
    r_values = bins*radius

    g, bins = np.histogram(dists, bins=r_values)
    g6, bins = np.histogram(dists, bins=r_values, weights=orders)
    bin_centers = (bins[1:] + bins[:-1]) / 2
    divisor = 2 * np.pi * bin_centers * (bins[1:]-bins[:-1]) * density * N

    return bin_centers, g/divisor, g6/divisor

def dists_and_orders(f, t=1000):
    idx = get_idx(f, t)
    dists = get_dists(f, idx)
    orders = get_orders(f, idx)
    return dists.ravel(), orders.ravel(), np.sum(idx)


def get_idx(f, t):
    return f.edge_distance.values > t
#     return f.x.values > 0


def get_dists(f, idx):
    x = f[['x', 'y']].values
    return spatial.distance.cdist(x[idx, :], x)


def get_orders(f, idx):
    orders = make_complex(f)
    order_grid = make_order_grid(orders, idx)
    return np.abs(order_grid)


def make_order_grid(orders, idx):
    orders = orders.reshape(-1, 1)
    return orders[idx] @ np.conj(orders).transpose()


def make_complex(f):
    return f['hexatic_order'].values


def flat_array(x):
    return np.concatenate([item.ravel() for item in x])

def add_edge_distance(data):
    points = data[['x', 'y']].values
    hull = spatial.ConvexHull(points)
    hull_points = points[hull.vertices, :]
    polygon = Polygon(hull_points)
    multi_point = MultiPoint(points)
    dists = [polygon.exterior.distance(p) for p in multi_point.geoms]
    data['edge_distance'] = dists
    return data, hull.volume

In [26]:
def get_gr(points):
    points = points.T.copy()
    points.resize((points.shape[0]+1, points.shape[1]))
    # print(points)
    rdf = freud.density.RDF(1000, 200)
    box_ = box.Box(Lx=np.max(points[0, :]), Ly=np.max(points[1, :]))
    rdf.compute(system=(box_, points.T))
    gr, r = rdf.rdf, rdf.bin_centers
    return gr, r

In [270]:
def get_radius(points):
    tree = spatial.KDTree(points)
    dists, indices = tree.query(points, 4)
    diameter = np.mean(dists[dists != 0])
    return diameter/2


In [274]:
def get_corr_from_data(fname, rmin, rmax, dr):
    data = pd.read_hdf(fname)
    frame = data.loc[0].copy()
    radius = get_radius(frame[['x', 'y']].values)
    frame['r'] = radius
    frame, area = add_edge_distance(frame)
    N = len(frame)
    r, g, g6 = corr(frame, N/area, radius, rmin, rmax, dr)
    return r, g, g6


In [331]:
import filehandling

In [336]:
def get_average_corr_from_data(direc_name, rmin, rmax, dr):
    rs = []
    gs = []
    g6s = []
    first = True
    for file in filehandling.get_directory_filenames(f"{direc_name}/*.hdf5"):
        data = pd.read_hdf(file)
        frame = data.loc[0].copy()
        if first:
            radius = get_radius(frame[['x', 'y']].values)
            first = False
        frame['r'] = radius
        frame, area = add_edge_distance(frame)
        N = len(frame)
        r, g, g6 = corr(frame, N/area, radius, rmin, rmax, dr)
        rs.append(r)
        gs.append(g)
        g6s.append(g6)
    return rs[0], np.mean(gs, axis=0), np.mean(g6s, axis=0)


In [337]:
r_187_mean, g_1_87_mean, g6_1_87_mean = get_average_corr_from_data("/media/data/Data/BallBearing/HIPS/IslandExperiments/1,87mmRepeats", 1, 32, 0.01)

In [339]:
r_191_mean, g_1_91_mean, g6_1_91_mean = get_average_corr_from_data("/media/data/Data/BallBearing/HIPS/IslandExperiments/1,91mmRepeatsB", 1, 32, 0.01)

In [348]:
r_193_mean, g_1_93_mean, g6_1_93_mean = get_average_corr_from_data("/media/data/Data/BallBearing/HIPS/IslandExperiments/1,93mmRepeats", 1, 32, 0.01)

In [352]:
r_196_mean, g_1_96_mean, g6_1_96_mean = get_average_corr_from_data("/media/data/Data/BallBearing/HIPS/IslandExperiments/1,96mmRepeats", 1, 32, 0.01)

In [354]:
plot_g6_over_g((r_196_mean, g_1_96_mean, g6_1_96_mean, '1.96mm'), (r_187_mean, g_1_87_mean, g6_1_87_mean, '1.87mm'))

<ipython-input-327-6fe53c191329>:11: RuntimeWarning: invalid value encountered in true_divide
  plt.plot(r/diameter, g6/g, label=key)


In [343]:
plt.plot(r_191_mean, g_1_91_mean)
plt.plot(r_191_mean, g6_1_91_mean)

In [288]:
def calculate_diameter_from_corr(r, g):
    diameter = r[np.argmax(g)]
    return diameter

In [275]:
r_1_91, g_1_91, g6_1_91 = get_corr_from_data("/media/data/Data/BallBearing/HIPS/IslandExperiments/1,91mmRepeatsB/19400010.hdf5", 1, 32, 0.01)

In [285]:
plt.plot(r_1_91, g_1_91)

In [279]:
r_1_96, g_1_96, g6_1_96 = get_corr_from_data("/media/data/Data/BallBearing/HIPS/IslandExperiments/1,96mmRepeats/19350013.hdf5", 1, 32, 0.01)

In [286]:
plt.plot(r_1_96, g_1_96)

In [281]:
r_1_87, g_1_87, g6_1_87 = get_corr_from_data("/media/data/Data/BallBearing/HIPS/IslandExperiments/1,87mmRepeats/19370064.hdf5", 1, 32, 0.01)

In [287]:
plt.plot(r_1_87, g_1_87)

## Compare to fake grid

In [289]:
diameter_1_87 = calculate_diameter_from_corr(r_1_87, g_1_87)
diameter_1_91 = calculate_diameter_from_corr(r_1_91, g_1_91)
diameter_1_91, diameter_1_87

(20.000985439448915, 20.029782383555364)

In [102]:
import trigrid

In [290]:
def calculate_perfect_corr(diameter):
    grid, area = make_grid(diameter, 40)
    r, g, g6 = corr(grid, len(grid)/area, diameter/2, 1, 32, 0.01)
    return r, g, g6

def make_grid(diameter, N):
    grid = trigrid.grid(diameter, N)
    grid = pd.DataFrame({'x': grid[0, :], 'y': grid[1, :],
                        'hexatic_order': 1+0j})
    grid['r'] = diameter / 2
    grid, area = add_edge_distance(grid)
    return grid, area

def calculate_corr(data, area, radius, nr_min=2, nr_max=16, dnr=0.01):
    r, g, g6 = corr(data, len(data)/area, radius, nr_min, nr_max, dnr)
    return r, g, g6

In [ ]:
def make_grid_with_noise(diameter, N, sigma):
    grid = trigrid.grid(diameter, N)
    noise = np.random.normal(scale=sigma, size=grid.shape)
    grid = grid + noise
    grid = pd.DataFrame({'x': grid[0, :], 'y': grid[1, :],
                        'hexatic_order': 1+0j})
    grid['r'] = diameter / 2
    grid, area = add_edge_distance(grid)
    return grid, area

def calculate_noisy_corr(diameter, noise):
    grid, area = make_grid_with_noise(diameter, 40, noise)


In [449]:
r_perfect, g_perfect, g6_perfect = calculate_perfect_corr(diameter_1_87)
plt.plot(r_perfect/r_perfect[np.argmax(g_perfect)], g_perfect)

In [305]:
def plot_cors(*sets, log=False):
    fig, ax = plt.subplots()
    for (r, g, key) in sets:
        diameter = r[np.argmax(g)]
        plt.plot(r/diameter, g/np.max(g), label=key)
    if log:
        ax.set_xscale('log')
        ax.set_yscale('log')
    plt.legend()
    plt.xlabel('r/d')
    plt.ylabel('g(r) scaled')

In [327]:
def plot_g6_over_g(*sets, log=False):
    fig, ax = plt.subplots()
    for (r, g, g6, key) in sets:
        diameter = r[np.argmax(g)]

        if key == 'grid':
            g6_over_g = g6/g
            for x in r[g6_over_g > 0]:
                plt.axvline(x/diameter, c='r', ls='--', alpha=0.5)
        else:
            plt.plot(r/diameter, g6/g, label=key)
    if log:
        ax.set_xscale('log')
        ax.set_yscale('log')
    plt.legend()
    plt.xlabel('r/d')
    plt.ylabel('$g_6/g$')

In [315]:
plot_cors((r_perfect, g6_perfect/g_perfect, 'grid'), (r_1_87, g6_1_87/g_1_87, '1.87mm'), (r_1_91, g6_1_91/g_1_91, '1.91mm'))

<ipython-input-315-696cb5975cd7>:1: RuntimeWarning: invalid value encountered in true_divide
  plot_cors((r_perfect, g6_perfect/g_perfect, 'grid'), (r_1_87, g6_1_87/g_1_87, '1.87mm'), (r_1_91, g6_1_91/g_1_91, '1.91mm'))


In [330]:
plot_g6_over_g((r_perfect, g_perfect, g6_perfect, 'grid'), (r_1_96, g_1_96, g6_1_96, '1.96mm'))

<ipython-input-327-6fe53c191329>:7: RuntimeWarning: invalid value encountered in true_divide
  g6_over_g = g6/g
<ipython-input-327-6fe53c191329>:11: RuntimeWarning: invalid value encountered in true_divide
  plt.plot(r/diameter, g6/g, label=key)


In [294]:
plt.plot(r_perfect, g_perfect/np.max(g_perfect))
plt.plot(r_1_87, g_1_87/np.max(g_1_87))

## Check the integer values of d

In [411]:
def plot_integer_values(*sets):
    for (r, g, g6, label) in sets:
        diameter = r[np.argmax(g)]
        r_over_d = r / diameter
        ri = [np.argmin(np.abs(r_over_d - i)) for i in range(1, 17)]
        # plt.plot(r_over_d[ri], g[ri], label=label)
        plt.loglog(r_over_d, g/np.max(g), '--', label=label)
        plt.loglog(r_over_d, 1/r_over_d)
    plt.xlabel('r/d')
    plt.ylabel('$g_6/g$')
    plt.legend()

In [412]:
plot_integer_values(
    (r_187_mean, g_1_87_mean, g6_1_87_mean, '1.87mm'),
    (r_191_mean, g_1_91_mean, g6_1_91_mean, '1.91mm'),
    (r_193_mean, g_1_93_mean, g6_1_93_mean, '1.93mm'),
    (r_196_mean, g_1_96_mean, g6_1_96_mean, '1.96mm')
)

In [402]:
plt.plot(r_perfect/r_perfect[np.argmax(g_perfect)], g_perfect/np.max(g_perfect))

In [470]:
r_int_perf = [1, 2, 3, 4, 5, 6, 7, 8, 9]
g_int_perf = [1, 0.468, 0.33, 0.24, 0.20, 0.18, 0.158, 0.136, 0.12]
plt.loglog(r_int_perf, g_int_perf)

In [404]:
plt.plot(np.log(r_int_perf), np.log(g_int_perf))

In [416]:
plt.plot(r_perfect[g_perfect>0], g_perfect[g_perfect>0])

In [450]:
perfect_bin_centers = r_perfect[g_perfect>0]
bin_edges = (perfect_bin_centers[:-1] + perfect_bin_centers[1:])/2
bin_edges = np.array([2*perfect_bin_centers[0] - bin_edges[0]] + bin_edges.tolist() + [2*perfect_bin_centers[-1]-bin_edges[-1]])
perfect_bin_centers.shape, bin_edges.shape

((86,), (87,))

In [430]:
def get_average_corr_from_data_bins(direc_name, rmin, rmax, dr, bins):
    rs = []
    gs = []
    g6s = []
    first = True
    for file in filehandling.get_directory_filenames(f"{direc_name}/*.hdf5"):
        data = pd.read_hdf(file)
        frame = data.loc[0].copy()
        if first:
            radius = get_radius(frame[['x', 'y']].values)
            first = False
        frame['r'] = radius
        frame, area = add_edge_distance(frame)
        N = len(frame)
        r, g, g6 = corr_custom_bins(frame, bins, N/area, radius, rmin, rmax, dr)
        rs.append(r)
        gs.append(g)
        g6s.append(g6)
    return rs[0], np.mean(gs, axis=0), np.mean(g6s, axis=0)

In [475]:
r_perfect, g_perfect, g6_perfect = calculate_perfect_corr(diameter_1_87)
perfect_bin_centers = r_perfect[g_perfect>0]
bin_edges = (perfect_bin_centers[:-1] + perfect_bin_centers[1:])/2
bin_edges = np.array([2*perfect_bin_centers[0] - bin_edges[0]] + bin_edges.tolist() + [2*perfect_bin_centers[-1]-bin_edges[-1]])
r, g, g6 = get_average_corr_from_data_bins("/media/data/Data/BallBearing/HIPS/IslandExperiments/1,87mmRepeats", 1, 16, 0.01, bin_edges/bin_edges[1])

In [474]:
x = r/r[np.argmax(g)]
plt.loglog(x, g/np.max(g))
plt.loglog(x, 1/x)